In [ ]:
region_bed = "../final/neighborhoods/latest/final_renamed.bed"
gene_annotation = "../final/annotation/latest/all_accesssions.hodgepodgemerged.gff3"
te_annotation = "../final/TE_annotation/latest/pangenome_TEannotation.gff3"

In [ ]:
library(stringr)
library(tidyverse)
library(gggenes)
options(repr.plot.width=8, repr.plot.height=6, repr.plot.res = 320)

In [ ]:
regions = read_tsv(region_bed, col_names = c("chrom", "start", "end", "region_name")) %>%
    mutate(accession=sub("(at\\d+)_.*", "\\1", chrom), nh_offset=pmax(1, start-100))

In [ ]:
genes = read_tsv(gene_annotation, col_names=c("chrom", "source", "type", "start", "end", "score", "strand", "phase", "attrs"), comment="#")  %>%
    filter(type%in%c("gene", "pseudogene", "pseuogenic_region", "transposable_element_gene")) %>%
    mutate(geneid=sub("ID=([^;]+).*", "\\1", attrs, perl=T))

In [ ]:
#ntk = read_tsv("../domainly/output/all_nlrtracker.tsv") %>%
ntk = read_csv("https://docs.google.com/spreadsheets/d/1zDNJx06aHCdjKWWai-mqQPEE3xmoJRTFNKn3d3zNUSo/export?gid=681416978&format=csv") %>%
    janitor::clean_names() %>%
    transmute(
        gene=id,
        gene_type=curated_type_final,
    ) %>%
    filter(gene_type == "nlr") %>%
    unique()

In [ ]:
gene_classes = genes %>%
    left_join(ntk, by=c("geneid"="gene")) %>%
    mutate(
        class = case_when(
            grepl("transposable", type) ~ "TE gene",
            grepl("pseudogen", type) & gene_type == "nlr"  ~ "NLR pseudogene",
            grepl("pseudogen", type) & is.na(gene_type)  ~ "pseudogene",
            type == "gene" & gene_type == "nlr" ~ "NLR",
            type == "gene" & is.na(gene_type) ~ "gene",
        )
    ) %>%
    select(geneid, class) %>%
    unique()

In [ ]:
gene_classes %>%
    count(geneid) %>%
    filter(n>1)

In [ ]:
colourscheme = tribble(
    ~class, ~colour,
    "gene", "#1F78B4",
    "NLR", "chocolate1",
    "NLR pseudogene", "#E31A1C",
    "pseudogene", "pink",
    "TE", "#555555",
    "TE gene", "#222222",
)
pal.colour=colourscheme$colour
names(pal.colour) = colourscheme$class

In [ ]:
nh_genes = regions %>%
    left_join(genes, by=join_by(chrom, start<=start, end>=end), suffix = c(".region", ".gene"), relationship = "many-to-many" ) %>%
    mutate(accession=sub("(at\\d+)_.*", "\\1", chrom)) %>%
    filter(!grepl("at6137", chrom))  %>%
    unique() %>%
    write_tsv("output/nh_genes-2024-04-19.tsv", na="")

In [ ]:
te = read_tsv(te_annotation, col_names=c("chrom", "source", "type", "start", "end", "score", "strand", "phase", "attrs"), comment = "#") %>%
    filter(!grepl("Parent", attrs)) %>%
    mutate(teid=sub("ID=([^;]+).*", "\\1", attrs, perl=T)) %>%
    select(-attrs, -score, -phase, -source, -type)

In [ ]:
nh_te = regions %>%
    left_join(te, by=join_by(chrom, start<=start, end>=end), suffix = c(".region", ".gene"), relationship = "many-to-many" )  %>%
    filter(!grepl("at6137", chrom))  %>%
    mutate(accession=sub("(at\\d+)_.*", "\\1", chrom)) %>%
    unique() %>%
    write_tsv("output/nh_te-2024-04-19.tsv", na="")

In [ ]:
nodes = read_tsv("../pangenome_pggb/km_diffent/output/node_radii_genomecoord.tsv.xz")

In [ ]:
nodes = nodes %>%
    mutate(chrom=sub(":.*", "", path)) %>%
    select(-path)

In [ ]:
reg_nodes = regions %>%
    left_join(nodes, by=join_by(chrom, start<end, end>start), suffix = c(".nh", ".node")) %>%
    mutate(startO = start.node-start.nh, endO=end.node-start.nh)

In [ ]:
outd = "output/2024-08-06_nh"
dir.create(outd, recursive = T, showWarnings = F)
nh_name = c("chr4_nbh12")
reg = regions %>%
    filter(region_name %in% nh_name) %>%
    filter(!grepl("at6137", chrom))  %>%
    mutate(startO=start-nh_offset, endO=end-nh_offset)
nhg = nh_genes %>%
    filter(region_name %in% nh_name) %>%
    mutate(startO=start.gene-nh_offset, endO=end.gene-nh_offset) %>%
    left_join(gene_classes, by=join_by(geneid)) 
nht = nh_te  %>%
    filter(region_name %in% nh_name) %>%
    mutate(startO=start.gene-nh_offset, endO=end.gene-nh_offset, class="TE")
nhr = reg_nodes %>%
    filter(region_name %in% nh_name, !grepl("at6137", chrom)) %>%
    mutate(x=round((endO+startO)/2/50)*50) %>%
    group_by(chrom, start.nh, end.nh, region_name, x) %>%
    mutate(mean_nr=sum((endO-startO)*node_diameter)/sum((endO-startO))) %>%
    left_join(
        nhg %>%
            select(chrom, startO, endO, class),
        by=join_by(chrom, startO<endO, endO>startO),
        suffix=c("", ".gene")
    ) %>%
    mutate(class=ifelse(is.na(class), "Intergenic", class))

p = ggplot(nhg, aes(xmin = startO, xmax = endO)) +
    #annotate("segment", x=reg$startO, xend=reg$endO, y=-10, yend=-10, colour="grey") +
    geom_gene_arrow(aes(xmin = startO, xmax = endO, forward=strand=="+", colour=class, fill=class, group=accession), y=-80, arrowhead_height = unit(.6, "mm"), arrowhead_width = unit(.4, "mm"),arrow_body_height=unit(1.5, "mm"), alpha=1, data=nhg)  +
    #geom_gene_arrow(aes(forward=strand=="+", colour=class, fill=class), y=-40, arrowhead_height = unit(.3, "mm"), arrowhead_width = unit(.4, "mm"), arrow_body_height=unit(1, "mm"), data=nht, alpha=0.5) +
    geom_point(aes(x=x, y=mean_nr,colour=class), size=0.2, data=nhr) +
    scale_y_continuous(breaks = c(0, 250), limits = c(-120, 260)) +
    scale_x_continuous(labels = scales::label_number(suffix = "Kbp", scale = 1e-3), breaks = seq(0, 300000, 50000)) + 
    scale_fill_manual(values=c(pal.colour, "Intergenic"="grey60"), guide=guide_legend(nrow =1), name="Gene Type", aesthetics = c("fill", "colour")) +
    facet_grid(accession~region_name, scale="free_x", space="free_x") +
    labs(y="Local Complexity", x="Neighbourhood Coordinate (bp)") +
    theme_classic() +
    theme(
        legend.position="bottom",
        strip.text.y.right=element_text(angle=0, hjust=0, vjust=0.5, size=8),
    )   

nh_name="fig3_rpp45"
#print(p)
ggsave(sprintf("%s/%s.png", outd, nh_name), width=7, height=7)
ggsave(sprintf("%s/%s.svg", outd, nh_name), width=7, height=7)
saveRDS(p, sprintf("%s/%s.rds", outd, nh_name))

In [ ]:
for (nh_name in unique(regions$region_name)) {
    reg = regions %>%
        filter(region_name %in% nh_name) %>%
        filter(!grepl("at6137", chrom))  %>%
        mutate(startO=start-nh_offset, endO=end-nh_offset)
    nhg = nh_genes %>%
        filter(region_name %in% nh_name) %>%
        mutate(startO=start.gene-nh_offset, endO=end.gene-nh_offset) %>%
        left_join(gene_classes, by=join_by(geneid)) 
    nht = nh_te  %>%
        filter(region_name %in% nh_name) %>%
        mutate(startO=start.gene-nh_offset, endO=end.gene-nh_offset, class="TE")
    nhr = reg_nodes %>%
        filter(region_name %in% nh_name, !grepl("at6137", chrom)) %>%
        mutate(x=round((endO+startO)/2/50)*50) %>%
        group_by(chrom, start.nh, end.nh, region_name, x) %>%
        mutate(mean_nr=sum((endO-startO)*node_diameter)/sum((endO-startO))) %>%
        left_join(
            nhg %>%
                select(chrom, startO, endO, class),
            by=join_by(chrom, startO<endO, endO>startO),
            suffix=c("", ".gene")
        ) %>%
        mutate(class=ifelse(is.na(class), "Intergenic", class))
    
    p = ggplot(nhg, aes(xmin = startO, xmax = endO)) +
        #annotate("segment", x=reg$startO, xend=reg$endO, y=-10, yend=-10, colour="grey") +
        geom_gene_arrow(aes(xmin = startO, xmax = endO, forward=strand=="+", colour=class, fill=class, group=accession), y=-160,  arrowhead_height = unit(.6, "mm"), arrowhead_width = unit(.4, "mm"),arrow_body_height=unit(1.5, "mm"), alpha=1, data=nhg)  +
        geom_gene_arrow(aes(forward=strand=="+", colour=class, fill=class), y=-160, arrowhead_height = unit(.3, "mm"), arrowhead_width = unit(.4, "mm"), arrow_body_height=unit(1, "mm"), data=nht, alpha=0.5) +
        geom_point(aes(x=x, y=mean_nr,colour=class), size=0.2, data=nhr) +
        scale_y_continuous(breaks = c(0, 250), limits = c(-220, 260)) +
        scale_x_continuous(labels = scales::label_number(suffix = " Kbp", scale = 1e-3)) + 
        scale_fill_manual(values=c(pal.colour, "Intergenic"="grey60"), guide=guide_legend(nrow =1), name="Gene Type", aesthetics = c("fill", "colour")) +
        facet_grid(accession~region_name, scale="free_x", space="free_x") +
        labs(y="Local Complexity", x="Neighbourhood Coordinate (bp)") +
        theme_classic() +
        theme(
            legend.position="bottom",
            strip.text.y.right=element_text(angle=0, hjust=0, vjust=0.5, size=8),
        )   
    
#    p=ggplot(nhg, aes(xmin = startO, xmax = endO, y = accession)) +
#        annotate("segment", x=reg$startO, xend=reg$endO, y=reg$accession, yend=reg$accession, colour="grey") +
#        geom_gene_arrow(aes(xmin = startO, xmax = endO, y =I(-10), forward=strand=="+", colour=class, fill=class, group=accession), arrowhead_height = unit(.8, "mm"), arrowhead_width = unit(.4, "mm"), alpha=1, data=nhg)  +
#        geom_gene_arrow(aes(forward=strand=="+", colour=class, fill=class), arrowhead_height = unit(1, "mm"), arrowhead_width = unit(.6, "mm"), alpha=0.7) +
#        geom_gene_arrow(aes(forward=strand=="+", colour=class, fill=class), arrowhead_height = unit(.5, "mm"), arrowhead_width = unit(.6, "mm"), arrow_body_height=unit(1.4, "mm"), data=nht, alpha=0.5) +
#        theme_genes()+
#        labs(y=NULL, x="Neighbourhood Coordinate (bp)", title=nh_name) +
#        theme(
#            legend.position="bottom", 
#            panel.grid.major.y = element_blank()
#        )
#
    print(p)
    ggsave(sprintf("%s/%s.png", outd, nh_name), width=7, height=7)
    ggsave(sprintf("%s/%s.svg", outd, nh_name), width=7, height=7)
    saveRDS(p, sprintf("%s/%s.rds", outd, nh_name))
}

In [ ]:
og = read_csv("https://docs.google.com/spreadsheets/d/1zDNJx06aHCdjKWWai-mqQPEE3xmoJRTFNKn3d3zNUSo/export?format=csv&gid=681416978") %>%
    janitor::clean_names() %>%
    filter(curated_type_final=="nlr") %>%
    transmute(geneid=id, fine_og= fine_orthogroups_second_draft_70_cd_hit_genes_and_pseudogenes, broad_og=broad_orthogroups) %>%
    unique() %>%
    glimpse()

In [ ]:
nh_name = c("chr1_nbh25")
acc_order = c("at9578", "at6929", "at9503", "at9883", "at9104", "at8285", "at9806", "at9762", "at9852", "at9900", "at7143", "at6923", "at9830","at9744",  "at9336", "at9847", "at9879")
reg = regions %>%
    filter(region_name %in% nh_name) %>%
    filter(!grepl("at6137", chrom))  %>%
    mutate(startO=start-nh_offset, endO=end-nh_offset)
nh_len= reg %>%
    transmute(chrom, region_name, nhl = endO-startO)
    
nhg = nh_genes %>%
    filter(region_name %in% nh_name) %>%
    mutate(startO=start.gene-nh_offset, endO=end.gene-nh_offset) %>%
    left_join(gene_classes, by=join_by(geneid))  %>%
    left_join(og, by=join_by(geneid)) %>%
    mutate(
        fillclass=case_when(
            class=="NLR" ~ sprintf("NLR OG %d", as.numeric(as.factor(as.character(fine_og)))),
            T ~ class,
        ),
    )
nht = nh_te  %>%
    filter(region_name %in% nh_name) %>%
    mutate(startO=start.gene-nh_offset, endO=end.gene-nh_offset, class="TE")

p=ggplot(nhg, aes(xmin = startO, xmax = endO, y = accession)) +
    annotate("segment", x=reg$startO, xend=reg$endO, y=reg$accession, yend=reg$accession, colour="grey") +
    geom_gene_arrow(aes(xmin = startO, xmax = endO, forward=strand=="+", fill=fillclass, group=accession), arrowhead_height = unit(.8, "mm"), arrowhead_width = unit(.4, "mm"), alpha=1, data=nhg)  +
    geom_gene_arrow(aes(forward=strand=="+"), arrowhead_height = unit(.5, "mm"), arrowhead_width = unit(.6, "mm"), arrow_body_height=unit(1.4, "mm"), fill="grey60", data=nht, alpha=0.5) +
    scale_fill_brewer(palette = "Paired", name = "Gene\nClass") +
    scale_y_discrete(limits=acc_order) +
    theme_genes()+
    labs(y=NULL, x="Neighbourhood Coordinate (bp)") +
    theme(
        legend.position="bottom", 
        panel.grid.major.y = element_blank()
    )

nh_name="fig5_chr1_nbh25"
print(p)
ggsave(sprintf("%s/%s.png", outd, nh_name), width=7, height=5)
ggsave(sprintf("%s/%s.svg", outd, nh_name), width=7, height=5)
saveRDS(p, sprintf("%s/%s.rds", outd, nh_name))

In [ ]:
nhg %>%
    select(fine_og, fillclass) %>%
    filter(!is.na(fine_og)) %>%
    unique()
    

# For figure 1

A good example

In [ ]:
nh_name = "chr5_nbh23"

syntanch = nh_genes %>%
    filter(region_name == nh_name) %>%
    group_by(accession) %>%
    summarise(left=first(geneid), right=last(geneid))  %>%
    pivot_longer(c(left, right)) %>%
    glimpse()
nhg = genes %>%
    mutate(accession=sub("(at\\d+)_.*", "\\1", chrom)) %>%
    filter(!grepl("at6137", chrom), grepl("chr5", chrom))  %>%
    left_join(gene_classes, by=join_by(geneid))  %>%
    left_join(
        read_csv("https://docs.google.com/spreadsheets/d/1zDNJx06aHCdjKWWai-mqQPEE3xmoJRTFNKn3d3zNUSo/export?gid=681416978&format=csv", na=c("NA","#N/A")) %>%
        janitor::clean_names() %>%
        select(geneid=id, fine_og=fine_orthogroups_second_draft_70_cd_hit_genes_and_pseudogenes, broad_og=combined_broad_orthogroups_and_pseudogenes) 
    ) %>%
    mutate(
        class=case_when(
            fine_og%in% c("OG0002656-cluster-1")  | fine_og == "OG0029600-cluster-1" & chrom == "at9104_1_chr5" ~ "FocalNLR",
            geneid %in% syntanch$value ~ "SyntenicAnchor",
            class == "gene" ~ "Gene",
            class == "TE gene" ~ "Gene",
            class == "NLR pseudogene" ~ "NLR",
            class == "pseudogene" ~ "Gene",
            T ~ class
        )) %>%
    group_by(chrom) %>%
    mutate(offset=min(start[class=="FocalNLR"])) %>%
    ungroup() %>%
    mutate(startO=start-offset, endO=end-offset) %>% 
    filter(startO> -75000 & endO < 75000)
reg = regions %>%
    filter(region_name == nh_name) %>%
    filter(!grepl("at6137", chrom)) %>%
    left_join(nhg %>%
                select(accession, offset) %>%
                unique()) %>%
    mutate(startO=start-offset, endO=end-offset) %>%
    glimpse()
nhg =  nhg %>%
    mutate(in_nh = purrr::pmap_lgl(list(startO, endO, accession), function(s, e, a) e < reg[reg$accession == a, "endO"] & s < reg[reg$accession == a, "endO"] & e > reg[reg$accession == a, "startO"] & s > reg[reg$accession == a, "startO"]  )) %>%
    glimpse()
                                    

In [ ]:
nhg %>% 
    count(class)

In [ ]:
p=ggplot(nhg, aes(xmin = startO, xmax = endO, y = accession)) +
    annotate("segment", x=-80000, xend=80000, y=reg$accession, yend=reg$accession, colour="grey") +
    annotate("segment", x=reg$startO, xend=reg$endO, y=reg$accession, yend=reg$accession, colour="black", size=2) +
    geom_gene_arrow(aes(forward=strand=="+", colour=class, fill=class, alpha=in_nh), arrowhead_height = unit(1, "mm"), arrowhead_width = unit(.6, "mm")) +
    scale_fill_manual(values=c(pal.colour, "Gene"="#1F78B4", "FocalNLR"="yellow2", "SyntenicAnchor"="maroon1"), guide=guide_legend(nrow =1), name="Gene Type", aesthetics = c("fill", "colour")) +
    scale_alpha_manual(values=c(.5, 1), guide=guide_none()) +
    scale_y_discrete(limits=rev) +
    theme_genes() +
    labs(y=NULL, x="Relative Coordinate (bp)") +
    theme(
        legend.position="bottom", 
        panel.grid.major.y = element_blank()
    )

print(p)
ggsave(sprintf("%s/Fig1_neighbourhood-syntenic-anchor-explanation.png", outd), width=7, height=4)
ggsave(sprintf("%s/Fig1_neighbourhood-syntenic-anchor-explanation.svg", outd), width=7, height=4)
saveRDS(p, sprintf("%s/Fig1_neighbourhood-syntenic-anchor-explanation2.rds", outd))

In [ ]:
save(nhg, reg, p, file=sprintf("%s/Fig1_neighbourhood-syntenic-anchor-explanation.rds", outd), compress=F)